In [ ]:
import tmdbsimple as tmdb
import pandas as pd
from tqdm.notebook import tqdm_notebook
import json

In [ ]:
with open('api.json', 'r') as file:
    login = json.load(file)
tmdb.API_KEY = login['api-key']

In [ ]:
titles = pd.read_csv("Data/refined_basics.csv", low_memory=False)
titles

In [ ]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for country in releases['countries']:
        if country['iso_3166_1'] == 'US':
            info['certification'] = country['certification']
    return info

In [ ]:
get_movie_with_rating(601)

In [ ]:
test_ids = titles['tconst']

In [ ]:
results = []
budget = []
revenue = []
certification = []
errors = []

In [ ]:
for movie_id in tqdm_notebook(test_ids):
    try:
        movie_info = get_movie_with_rating(movie_id)
        budget.append(movie_info['budget'])
        revenue.append(movie_info['revenue'])
        certification.append(movie_info['certification'])
    except Exception as e:
        errors.append([movie_id, e])

In [ ]:
results_series = pd.Series(results)

In [ ]:
budget_series = pd.Series(budget)

In [ ]:
revenue_series = pd.Series(revenue)

In [ ]:
certification_series = pd.Series(certification)

In [ ]:
titles['budget'] = budget_series

In [ ]:
titles['revenue'] = revenue_series

In [ ]:
titles['certification'] = certification_series

In [ ]:
titles

In [ ]:
titles.to_csv("financials_included_movies.csv")